# Coin Metrics API Example Endpoint Uses

Coin Metrics offers a broad suite of data related to crypto, crypto markets, and crypto networks. This data is all available via the Coin Metrics API, which is accessible through a Python client. This notebook provides examples of how to use each endpoint available through the API. You can find full documentation and workflow demos in the Resources section below.

## Resources

* The [Coin Metrics Python API Client library](https://coinmetrics.github.io/api-client-python/site/index.html) includes information on the Python API Client.
* The [Coin Metrics API v4](https://docs.coinmetrics.io/api/v4) website contains the full set of endpoints and data offered by Coin Metrics.
* The [API Spec](https://coinmetrics.github.io/api-client-python/site/api_client.html) contains a full list of functions, documentation, and the syntax for the Python API client.
* The [Coin Metrics Data Encyclopedia](https://docs.coinmetrics.io/) gives detailed, conceptual explanations of the data and metrics that Coin Metrics offers.
* [Exploring Crypto Data Using the Coin Metrics Python API Client](https://gitlab.com/coinmetrics/data-delivery/api-client-python/-/blob/master/examples/notebooks/walkthrough_community_python_event.ipynb) is a jupyter notebook walkthrough of how to use the Coin Metrics API in data projects.

## Coin Metrics Research

* [Coin Metrics State of the Network](https://coinmetrics.substack.com/), our weekly data-driven newsletter highlighting on-chain (network) data
* [Coin Metrics State of the Market](https://coinmetrics.io/insights/state-of-the-market/), weekly report contextualizing the week’s crypto market movement
* [Other Original Research](https://coinmetrics.io/insights/original-research/) and Long-form Reports

##  Setup

In [ ]:
import datetime
from os import environ
import logging
import orjson

import numpy as np
import pandas as pd

from datatools.data import get_api_key_from_resources, read_submodule_file
from coinmetrics.api_client import CoinMetricsClient, CmStream

In [ ]:
# Pull API key from environemnt
try:
    api_key = environ["CM_API_KEY"]
    logging.info("Using API key found in environment")
except KeyError:
    api_key = ""
    logging.info("API key not found. Using community client")

# Instantiate client
client = CoinMetricsClient(api_key)

## Endpoints

### Catalog

The catalog endpoints will query metadata about the dimension specified in the endpoint. For example, to find the metadata related to a given asset or group of assets, the catalog_assets() API call will be useful. Examples for each catalog endpoint are given below.

In [ ]:
catalog_assets = client.catalog_assets(assets=["BTC"])

catalog_metrics = client.catalog_metrics(metrics=["ReferenceRateUSD"],
                                         reviewable=False)

catalog_pairs = client.catalog_asset_pairs(asset_pairs=['btc-eth'])

catalog_exchange_assets = client.catalog_exchange_assets(exchange_assets=["bibox-1inch"])

catalog_institutions = client.catalog_institutions(institutions="grayscale")

catalog_exchanges = client.catalog_exchanges(exchanges="binance")

# Catalog Markets - Syntax is rather specific here
catalog_markets = client.catalog_markets(markets=["binance-BTCUSDT-future"])

catalog_markets = client.catalog_markets(market_type="future",
                                         exchange="binance",
                                         symbol="BTCUSDT")

catalog_markets = client.catalog_markets(market_type="spot",
                                         exchange="binance",
                                         base='btc',
                                         quote='usdt')

catalog_market_metrics = client.catalog_market_metrics(markets=["binance-BTCUSDT-future"])

catalog_market_candles = client.catalog_market_candles(markets="ftx-btc-usdt-spot")

catalog_asset_alerts = client.catalog_asset_alerts(["btc"])

catalog_indexes = client.catalog_indexes(indexes="CMBI10")

### Full Catalog

The catalog endpoints above will only return metadata for queries reachable with the specified API key priviledges. To see the full catalog, including inaccessible data, use the full catalog endpoints.

In [ ]:
catalog_full_all = client.catalog_full_assets(assets=['btc'])

catalog_full_asset_pairs = client.catalog_full_asset_pairs(['btc-usdt'])

catalog_full_metrics = client.catalog_full_metrics(metrics=['TxCnt'])

catalog_full_exchange_assets = client.catalog_full_exchange_assets(exchange_assets=['binance.us-btc'])

catalog_full_institutions = client.catalog_full_institutions(institutions="grayscale")

catalog_full_exchanges = client.catalog_full_exchanges(exchanges=["bitfinex"])

catalog_full_markets = client.catalog_full_markets(markets=["binance.us-btc-usdt-spot"])

catalog_full_markets = client.catalog_full_markets(market_type="future",
                                                   exchange="binance",
                                                   symbol="BTCUSDT")

catalog_full_markets = client.catalog_full_markets(market_type="spot",
                                                   exchange="binance",
                                                   base='btc',
                                                   quote='usdt')

catalog_full_market_metrics = client.catalog_full_market_metrics(markets="binance-BTCUSDT-future")

catalog_full_market_candles = client.catalog_full_market_candles(markets="ftx-btc-usdt-spot")

catalog_full_asset_alerts = client.catalog_full_asset_alerts(["btc"])

catalog_full_indexes = client.catalog_full_indexes(indexes=["CMBI10"])

### Metrics

The catalog helps us see what data is availble to query, but it is now time to get the timeseries data. Timeseries metric data can be queried in the following ways, specifying by assets, pairs, exchanges, and exchange assets. The time frame and frequency can also be specified.

In [ ]:
asset_metrics = client.get_asset_metrics(assets=["BTC"],
                                         metrics=["ReferenceRateUSD"],
                                         start_time="2021-01-01",
                                         end_time="2022-02-16",
                                         frequency="1d",).to_dataframe()

pair_metrics = client.get_pair_metrics(pairs=["btc-usdt"],
                                       metrics=["open_interest_reported_future_perpetual_usd",
                                                "volume_reported_future_perpetual_usd_1d"],
                                       start_time='2022-01-01',
                                       end_time='2022-01-10').to_dataframe()

exchange_metrics = client.get_exchange_metrics(exchanges=["binance"],
                                               metrics=["open_interest_reported_future_perpetual_usd",
                                                        "volume_reported_future_perpetual_usd_1d"],
                                               start_time='2022-01-01',
                                               end_time='2022-01-10').to_dataframe()

exchange_asset_metrics = client.get_exchange_asset_metrics(exchange_assets=["ftx-sol"],
                                                           metrics=["volume_reported_future_usd_1d"],
                                                           start_time='2022-01-01',
                                                           end_time='2022-01-10').to_dataframe()

### Markets

Market data can also be queried in a similar way. Below are the standard market data timeseries endpoints.

In [ ]:
market_trades = client.get_market_trades(markets=['coinbase-btc-usd-spot'],
                                         start_time='2020-01-01',
                                         end_time='2020-01-03',
                                         limit_per_market=10).to_dataframe()

market_open_interest = client.get_market_open_interest(markets=["binance-BTCUSDT-future"]).to_dataframe()

market_liquidations = client.get_market_liquidations(markets=["binance-BTCUSDT-future"]).to_dataframe()

market_funding_rates = client.get_market_funding_rates(markets=["binance-BTCUSDT-future"]).to_dataframe()

market_order_books = client.get_market_orderbooks(markets="coinbases-btc-usd-spot",
                                                  page_size=10,
                                                  start_time="2022-04-27T00:00:00.000000000",
                                                  end_time="2022-04-27T01:00:00.000000000",
                                                  depth_limit=10).to_dataframe()

market_quotes = client.get_market_quotes(markets=["binance-BTCUSDT-future"]).to_dataframe()

market_candles = client.get_market_candles(markets=["binance-btc-usdt-spot"],
                                           start_time='2020-01-01',
                                           end_time='2022-01-03').to_dataframe()

market_greeks = client.get_market_greeks(markets=["deribit-BTC-10APR21-50000-P-option"]).to_dataframe()

### Indices

The full suite of indexes available through the API can be accessed through these endpoints.

In [ ]:
index_levels = client.get_index_levels(indexes=["CMBI10EX"],
                                       start_time='2020-01-01',
                                       end_time='2022-01-03').to_dataframe()

index_constituents = client.get_index_constituents(indexes=["CMBI10"]).to_dataframe()

### Farum

Alerts and other endpoints associated with Farum can be accessed as follows.

In [ ]:
asset_alerts = client.get_asset_alerts(assets=["eth"],
                                       alerts="block_count_by_unknown_miners_6b_hi").to_dataframe()

mining_pool_tips_summary = client.get_mining_pool_tips_summary(assets='btc',
                                                              page_size=50).to_dataframe()

mempool_feerates = client.get_mempool_feerates(assets='btc', 
                                               start_time='2022-04-28T00:00:00', 
                                               end_time='2022-04-28T00:01:00').to_dataframe()

### Stream

In [ ]:
stream_asset_metrics = client.get_stream_asset_metrics(assets=['btc', 'eth'], 
                                                       frequency='1s',
                                                       metrics=['ReferenceRateUSD', 
                                                                'ReferenceRateBTC', 
                                                                'ReferenceRateETH']
                                                      )

stream_market_trades = client.get_stream_market_trades(markets='binance-btc-usdt-spot')

stream_market_orderbooks = client.get_stream_market_orderbooks(markets="binance-btc-usdt-spot")

stream_market_quotes = client.get_stream_market_quotes(markets="binance-btc-usdt-spot")

These stream endpoints connect to the websocket API and can be used in the following way:

In [ ]:
stream = client.get_stream_asset_metrics(
    assets=['btc', 'eth'], frequency='1s',
    metrics=['ReferenceRateUSD', 'ReferenceRateBTC', 'ReferenceRateETH']
)

def on_message(
        stream: CmStream, message: str
) -> None:
    """
    Custom message callable to be passed in the streaming object
    :param stream: CmStream, WebSocketApp connection
    :param message: str, The message relayed by the API
    :return: None
    """
    data = orjson.loads(message)
    print(data)
    sequence_id = int(data['cm_sequence_id'])
    max_cm_sequence_id = 10 # close connection after 10 messages
    if sequence_id >= max_cm_sequence_id:
        print(f"Closing the connection after {max_cm_sequence_id} messages...")
        stream.close()

# blocks until connection is closed or interrupted
stream.run(on_message=on_message)

### Atlas 

Users can also query Atlas data using the following block search endpoints.

In [ ]:
list_of_blocks = client.get_list_of_blocks_v2(asset='ltc',
                                              start_time='2022-04-28T00:00:00', 
                                              end_time='2022-04-28T01:00:00',
                                              page_size=100).to_dataframe()

list_of_accounts = client.get_list_of_accounts_v2(asset='usdc',
                                                  start_time='2022-04-28T00:00:00', 
                                                  end_time='2022-04-28T01:00:00',
                                                  page_size=100).to_dataframe()

list_of_sub_accounts = client.get_list_of_sub_accounts_v2(asset='ltc',
                                                          start_time='2022-04-28T00:00:00', 
                                                          end_time='2022-04-28T01:00:00',
                                                          page_size=100).to_dataframe()

list_of_transactions = client.get_list_of_transactions_v2(asset='ltc',
                                                          start_time='2022-04-28T00:00:00', 
                                                          end_time='2022-04-28T01:00:00',
                                                          page_size=100).to_dataframe()

list_of_balance_updates = client.get_list_of_balance_updates_v2(asset='usdc',
                                                                start_time='2022-04-28T00:00:00', 
                                                                end_time='2022-04-28T01:00:00',
                                                                page_size=100).to_dataframe()